<a href="https://colab.research.google.com/github/Th3Mayar/exercises_datasetSQL/blob/main/exercises_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset

Imports libraries

In [ ]:
import sqlite3
import pandas as pd

Connect to an in-memory SQLite database (temporary for this example)

In [ ]:
conn = sqlite3.connect(":memory:")

Load the cities dataset

In [ ]:
url_cities = "https://raw.githubusercontent.com/datasets/world-cities/master/data/world-cities.csv"
cities = pd.read_csv(url_cities)
cities.to_sql('cities', conn, index=False, if_exists='replace')

28724

Load an employee dataset

In [ ]:
url_employees = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
employees = pd.read_csv(url_employees, sep="|")
employees.to_sql('employees', conn, index=False, if_exists='replace')

943

Country dataset (to relate to cities)

In [ ]:
# Dataset de países (para relacionarlo con cities)
countries_data = {
    'country': ['Andorra', 'United Arab Emirates', 'United States', 'Canada'],
    'region': ['Europe', 'Middle East', 'North America', 'North America'],
    'population_millions': [0.077, 9.77, 331, 37.59]
}

Country dataset (to relate to cities)

In [ ]:
countries = pd.DataFrame(countries_data)
countries.to_sql('countries', conn, index=False, if_exists='replace')

4

Department dataset (to relate to employees)

In [ ]:
departments_data = {
    'occupation': ['technician', 'writer', 'other', 'manager'],
    'department_name': ['Technical Support', 'Editorial', 'Miscellaneous', 'Management'],
    'budget_millions': [5.5, 3.0, 2.5, 10.0]
}

departments = pd.DataFrame(departments_data)
departments.to_sql('departments', conn, index=False, if_exists='replace')

4

Function to execute SQL queries and display the results in the notebook

In [ ]:
def run_query(query):
    return pd.read_sql_query(query, conn)

#Exercises (A)

##Count Cities by Country

In [ ]:
query_cities_count = """
SELECT country, COUNT(name) AS city_count
FROM cities
GROUP BY country
HAVING city_count > 2;
"""
print("Number of cities per country (more than 2):")
print(run_query(query_cities_count))

Number of cities per country (more than 2):
            country  city_count
0       Afghanistan          50
1           Albania          22
2           Algeria         292
3            Angola          95
4         Argentina         233
..              ...         ...
179        Viet Nam         118
180  Western Sahara           3
181           Yemen          23
182          Zambia          30
183        Zimbabwe          28

[184 rows x 2 columns]


##List of Employees and Departments

In [ ]:
query_employees_departments = """
SELECT e.user_id, e.occupation, d.department_name
FROM employees e
LEFT JOIN departments d ON e.occupation = d.occupation;
"""
print("nList of employees with their department:")
print(run_query(query_employees_departments))

nList of employees with their department:
     user_id     occupation    department_name
0          1     technician  Technical Support
1          2          other      Miscellaneous
2          3         writer          Editorial
3          4     technician  Technical Support
4          5          other      Miscellaneous
..       ...            ...                ...
938      939        student               None
939      940  administrator               None
940      941        student               None
941      942      librarian               None
942      943        student               None

[943 rows x 3 columns]


##Average Age by Occupation

In [ ]:
query_average_age = """
SELECT occupation, AVG(age) AS average_age
FROM employees
GROUP BY occupation;
"""
print("nAverage age by occupation:")
print(run_query(query_average_age))

nAverage age by occupation:
       occupation  average_age
0   administrator    38.746835
1          artist    31.392857
2          doctor    43.571429
3        educator    42.010526
4        engineer    36.388060
5   entertainment    29.222222
6       executive    38.718750
7      healthcare    41.562500
8       homemaker    32.571429
9          lawyer    36.750000
10      librarian    40.000000
11      marketing    37.615385
12           none    26.555556
13          other    34.523810
14     programmer    33.121212
15        retired    63.071429
16       salesman    35.666667
17      scientist    35.548387
18        student    22.081633
19     technician    33.148148
20         writer    36.311111


##Cities and Region

In [ ]:
query_cities_regions = """
SELECT c.name, co.region
FROM cities c
JOIN countries co ON c.country = co.country;
"""
print("nCities and their region:")
print(run_query(query_cities_regions))

nCities and their region:
                                       name         region
0                              les Escaldes         Europe
1                          Andorra la Vella         Europe
2                                   Warīsān    Middle East
3                                Umm Suqaym    Middle East
4                        Umm Al Quwain City    Middle East
...                                     ...            ...
3704                          Tustin Legacy  North America
3705  Aliamanu / Salt Lake / Foster Village  North America
3706        Mililani Mauka / Launani Valley  North America
3707                              Koolauloa  North America
3708      Makakilo / Kapolei / Honokai Hale  North America

[3709 rows x 2 columns]


#Exercises (B)

##Calculate the Number of Cities per Region with CTE

In [ ]:
query_cities_region_cte = """
WITH RegionCities AS (
    SELECT co.region, COUNT(c.name) AS city_count
    FROM countries co
    JOIN cities c ON co.country = c.country
    GROUP BY co.region
)
SELECT region, city_count
FROM RegionCities
WHERE city_count > 3;
"""
print("nNumber of cities per region (more than 3):")
print(run_query(query_cities_region_cte))

nNumber of cities per region (more than 3):
          region  city_count
0    Middle East          63
1  North America        3644


##Employee Age Ranking by Occupation with Window Functions

In [ ]:
query_employee_ranking = """
SELECT user_id, occupation, age,
       RANK() OVER (PARTITION BY occupation ORDER BY age DESC) AS age_rank
FROM employees
WHERE age IS NOT NULL
ORDER BY occupation, age_rank;
"""
print("nRanking of employees by age within their occupation:")
print(run_query(query_employee_ranking))

nRanking of employees by age within their occupation:
     user_id     occupation  age  age_rank
0        803  administrator   70         1
1        266  administrator   62         2
2        131  administrator   59         3
3        819  administrator   59         3
4          7  administrator   57         5
..       ...            ...  ...       ...
938      801         writer   22        41
939       50         writer   21        42
940      445         writer   21        42
941      507         writer   18        44
942      620         writer   18        44

[943 rows x 4 columns]


##Comparison of Cities and Population by Region

In [ ]:
query_cities_population = """
SELECT co.region, COUNT(c.name) AS city_count, SUM(co.population_millions) AS total_population
FROM countries co
JOIN cities c ON co.country = c.country
GROUP BY co.region
HAVING city_count > 3 AND total_population > 50;
"""
print("nRegions with more than 3 cities and a total population greater than 50 million:")
print(run_query(query_cities_population))

nRegions with more than 3 cities and a total population greater than 50 million:
          region  city_count  total_population
0    Middle East          63            615.51
1  North America        3644        1107578.24


##Employees Older than the Overall Average in Your Department

In [ ]:
query_employees_above_average_age = """
SELECT user_id, occupation, age
FROM employees e1
WHERE age > (SELECT AVG(age) FROM employees e2 WHERE e1.occupation = e2.occupation);
"""
print("nEmployees older than average in their occupation:")
print(run_query(query_employees_above_average_age))

nEmployees older than average in their occupation:
     user_id     occupation  age
0          2          other   53
1          6      executive   42
2          7  administrator   57
3          9        student   29
4         10         lawyer   53
..       ...            ...  ...
403      934       engineer   61
404      937       educator   48
405      938     technician   38
406      939        student   26
407      942      librarian   48

[408 rows x 3 columns]


#Exercises (C)

##Find the Countries with the Highest Number of Cities and their Region

In [ ]:
query_countries_most_cities = """
WITH CountryCityCount AS (
    SELECT co.country, co.region, COUNT(c.name) AS city_count
    FROM countries co
    JOIN cities c ON co.country = c.country
    GROUP BY co.country, co.region
),
RankedCountries AS (
    SELECT country, region, city_count,
           RANK() OVER (PARTITION BY region ORDER BY city_count DESC) AS city_rank
    FROM CountryCityCount
)
SELECT country, region, city_count
FROM RankedCountries
WHERE city_rank = 1;
"""
print("nCountries with the highest number of cities per region:")
print(run_query(query_countries_most_cities))

nCountries with the highest number of cities per region:
                country         region  city_count
0               Andorra         Europe           2
1  United Arab Emirates    Middle East          63
2         United States  North America        3308


##Comparison of Budgets and Number of Employees by Department

In [ ]:
query_budget_employee_comparison = """
SELECT d.department_name,
       COUNT(e.user_id) AS num_employees,
       d.budget_millions,
       (d.budget_millions / COUNT(e.user_id)) AS budget_per_employee
FROM departments d
LEFT JOIN employees e ON d.occupation = e.occupation
GROUP BY d.department_name
HAVING budget_per_employee < 0.1;
"""
print("nDepartments where the budget per employee is less than 0.1 million:")
print(run_query(query_budget_employee_comparison))

nDepartments where the budget per employee is less than 0.1 million:
  department_name  num_employees  budget_millions  budget_per_employee
0       Editorial             45              3.0             0.066667
1   Miscellaneous            105              2.5             0.023810
